In [1]:
from wigner_functions import *
import h5py
from scipy.sparse import *
import zarr

In [2]:
import time

In [3]:
lmax=200 #1e4
wlmax=5e1
lmax=np.int(lmax+1)
wlmax=np.int(wlmax+1)

In [4]:
ncpu=8
l_step=10 #not used with dask
w_l=np.arange(wlmax+1)
l=np.arange(lmax+1)

In [5]:
??LocalCluster

Object `LocalCluster` not found.


In [6]:
import dask
import dask.array as da
from dask import delayed

from distributed import LocalCluster
from dask.distributed import Client  # we already had this above
#http://distributed.readthedocs.io/en/latest/_modules/distributed/worker.html
LC=LocalCluster(n_workers=1,processes=False,memory_limit='10gb',threads_per_worker=8,memory_spill_fraction=.99,
               memory_monitor_interval='2000ms',scheduler_port=12234,diagnostics_port=8800)#dashboard_address='localhost:8787')
client=Client(LC,asynchronous=False)

In [7]:
client

Client Scheduler: tcp://127.0.0.1:12234 Dashboard: http://127.0.0.1:8800/status,Cluster Workers: 1 Cores: 8 Memory: 10.00 GB


In [8]:
def wigner_3j_asym(j_1,j_2,j_3,m_1,m_2,m_3): #assume j1,j2>>j3.... wiki
    sj=(j_1+j_2+1)
    th=np.arccos((m_1-m_2)/sj)
    wd=wigner_d(m_3,j_2-j_1,np.atleast_1d(th),j_3)[0,0]
    print(wd)
    return ((-1)**(j_2+m_2))*wd/np.sqrt(sj)

In [9]:
def wigner_3j_asym_H(j_1,j_2,j_3,m_1,m_2,m_3): #j1+j2+j3>>1, m1=m2=m3=0.. Hivon
    wj=2*(j1*j2)**2
    wj+=2*(j2*j3)**2
    wj+=2*(j1*j3)**2
    wj-=j1**4+j2**4+j3**4
    wj=wj**(-0.5)
    wj*=2./np.pi
    return np.sqrt(wj)*(-1)**((j_1+j_2+j_3)/2)

def wigner_3j_asym_H2(j_1,j_2,j_3,m_1,m_2,m_3): #j1+j2+j3>>1, m1=m2=m3=0.. Hivon
    J=j_1+j_2+j_3
    logwj=log_factorial(J/2)
    logwj-=log_factorial(J/2-j_1)
    logwj-=log_factorial(J/2-j_2)
    logwj-=log_factorial(J/2-j_3)
    logwj-=0.5*log_factorial(J+1)
    logwj+=0.5*log_factorial(J-2*j_1)
    logwj+=0.5*log_factorial(J-2*j_2)
    logwj+=0.5*log_factorial(J-2*j_3)
    return (-1)**(J/2)*np.exp(logwj)

In [10]:
def wig3j_map(m1,m2,m3,j1,j2,j3,asym_fact=np.inf):
#     n1=len(l)
#     n2=len(l)
#     n3=len(j3)

#     c=np.array(list(Comb(l,l,j3)))
# #     print(c.shape)
#     j_max=np.amax(l.max()+l.max()+l.max()+1)
#     _calc_factlist(j_max)
#     d_mat=client.gather(client.map(partial(wigner_3j_3, m_1, m_2, m_3),c))
#     d_mat=np.array(d_mat).reshape(n1,n2,n3)
    print('w3j_map doing ',np.atleast_1d(j3)[0])
    xx=Wigner3j_parallel( m1, m2, m3,np.atleast_1d(j1), np.atleast_1d(j2) ,np.atleast_1d(j3),ncpu=1,
                             asym_fact=asym_fact)

    return xx


# dst=client.map(wig3j_map,w_l)

In [11]:
def Wigner3j_parallel2( m_1, m_2, m_3,j_1, j_2, j_3,ncpu=None,asym_fact=np.inf):
    if ncpu is None:
        ncpu=cpu_count()-2
    p=Pool(ncpu)

    j_max=np.amax(j_1.max()+j_2.max()+j_3.max()+1)
#     _calc_factlist(j_max)

    n1=len(j_1)
    n2=len(j_2)
    n3=len(j_3)

    c=np.array(np.meshgrid(j_1,j_2,j_3,indexing='ij')).T.reshape(-1,3) #only needed to put cuts below. Otherwise Comb is better

    x=c[:,0]+c[:,1]-c[:,2]>=0
    x*=c[:,0]-c[:,1]+c[:,2]>=0
    x*=-c[:,0]+c[:,1]+c[:,2]>=0
    
    x*=abs(m_1) <= c[:,0]
    x*=abs(m_2) <= c[:,1]
    x*=abs(m_3) <= c[:,2]
    
    if m_1==0 and m_2==0 and m_3==0:
        x*=(c[:,0]+c[:,1]+c[:,2])%2==0

    c=c[x]
    
    
    d_mat=p.map(partial(wigner_3j_3,asym_fact, m_1, m_2, m_3),c,chunksize=100)
    p.close()
    
    dd=np.zeros((n1,n2,n3))
    indx1=np.searchsorted(j_1,c[:,0])
    indx2=np.searchsorted(j_2,c[:,1])
    indx3=np.searchsorted(j_3,c[:,2])
    dd[indx1,indx2,indx3]=d_mat
    
    return dd

In [12]:
asym_fact=100

In [13]:
# if lmax>500:
#     fname='temp/dask_wig3j_big_{i}.zarr'
# else:
#     fname='temp/dask_wig3j_test_{i}.zarr'
fname='temp/dask_wig3j_l{lmax}_w{wlmax}_{i}.zarr'
if asym_fact is not np.inf:
    fname='temp/dask_wig3j_l{lmax}_w{wlmax}_{i}_asym'+str(asym_fact)+'.zarr'

In [14]:
fname,lmax,wlmax

('temp/dask_wig3j_l{lmax}_w{wlmax}_{i}_asym100.zarr', 201, 51)

In [15]:
j3_b=np.arange(0,50+1,step=10)
j3_b[-1]=50
j3_b

array([ 0, 10, 20, 30, 40, 50])

In [38]:
from multiprocessing import Pool
def wig3j_recur_1d(j2s,m1,m2,m3,j3_outmax,j1):
    out_ij={}
    x=j2s<=j1
    for j2 in j2s[x]:
#         print('doing ',j1,j2)
        out_ij[j2]=wig3j_recur(j1,j2,m1,m2,m3,j3_outmax=j3_outmax)
#         print('done ',j1,j2)
    return out_ij

def wig3j_recur_2d(j1b,j2b,m1,m2,m3,j3_outmax,step,z1_out):
    try:
        out= np.zeros((min(step,lmax-j1b),min(step,lmax-j2b),j3_outmax))
    except Exception as err:
        print(err, step,lmax, j1b,j2b)
        crash 
    j1s=np.arange(j1b,min(lmax,j1b+step))
    j2s=np.arange(j2b,min(lmax,j2b+step))
    out_ij={}
    t1=time.time()
    funct=partial(wig3j_recur_1d, j2s,m1,m2,m3,j3_outmax)
    pool=Pool(10)
    out_ij=pool.map(funct,j1s)
    pool.close()
#     for j1 in np.arange(len(j1s)):
# #         out_ij[j1]=client.submit(wig3j_recur_1d,j1s[j1],j2s,m1,m2,m3,j3_outmax)
#         out_ij[j1]=wig3j_recur_1d(j1s[j1],j2s,m1,m2,m3,j3_outmax)
        
    t2=time.time()
    for j1 in np.arange(len(j1s)):
        for j2 in np.arange(len(j2s)):
            if j2s[j2]<=j1s[j1]:
                out[j1,j2,:]=out_ij[j1][j2s[j2]]
#                 z1[j1s[j1],j2s[j2],:]=out_ij[j1][j2s[j2]][0,0,:]
#                 z1[j2s[j2],j1s[j1],:]=out_ij[j1][j2s[j2]][0,0,:]

    t3=time.time()
    z1[j1b:j1b+step,j2b:j2b+step,:]=out
    z1[j2b:j2b+step,j1b:j1b+step,:]=out.transpose(1,0,2)
#     z1[j1b:j1b+step,j2b:j2b+step,:]=out
#     print('done all ',j1b,j2b,t3-t2)
#     except Exception as err:
#         print(err,out.shape,j1s.shape,j2s.shape,j1b,j2b,j2,j2s[j2],step,lmax)
    return 0


In [39]:
step=10
lb=np.arange(lmax,step=step)
lb[-1]=lmax

In [40]:
lb

array([  0,  10,  20,  30,  40,  50,  60,  70,  80,  90, 100, 110, 120,
       130, 140, 150, 160, 170, 180, 190, 201])

In [41]:
lmax

201

In [42]:
z1 = zarr.open(fname.format(i=str(2)+'j3m',lmax=lmax,wlmax=wlmax), mode='w', shape=(lmax,lmax,wlmax),
...                chunks=(lmax/10, lmax/10,wlmax/10),
               dtype='float32',overwrite=True)

In [43]:
step

10

In [44]:
client.restart()

Client Scheduler: tcp://127.0.0.1:12234 Dashboard: http://127.0.0.1:8800/status,Cluster Workers: 1 Cores: 8 Memory: 10.00 GB


In [45]:
m1=-2
m2=2
m3=0
t0=time.time()
for lb1 in lb:
    ww_out={}
    for lb2 in lb:
        print('doing',lb1,lb2)
        t1=time.time()
        ww_out[lb2]=wig3j_recur_2d(lb1,lb2,m1,m2,m3,wlmax,step,z1)
#     ww_out=client.compute(ww_out).result()
#         ww_out[lb2]=client.submit(wig3j_recur_2d,lb1,lb2,m1,m2,m3,wlmax,step,z1)
#         ww_out[lb2]=wig3j_recur_2d(lb1,lb2,m1,m2,m3,wlmax,step)
#     for lb2 in lb:
# #         z1[lb1:lb1+step,lb2:lb2+step,:]=ww_out[lb2].result()
#         ww_out[lb2]=ww_out[lb2].result()
#         t2=time.time()
#         print('done',lb1,lb2,t2-t1)
    del ww_out
    t2=time.time()
    print('done',lb1,t2-t1)
print('done all',t2-t0)

doing 0 0
doing 0 10
doing 0 20
doing 0 30
doing 0 40
doing 0 50
doing 0 60
doing 0 70
doing 0 80
doing 0 90
doing 0 100
doing 0 110
doing 0 120
doing 0 130
doing 0 140
doing 0 150
doing 0 160
doing 0 170
doing 0 180
doing 0 190
doing 0 201
done 0 0.17795777320861816
doing 10 0
doing 10 10
doing 10 20
doing 10 30
doing 10 40
doing 10 50
doing 10 60
doing 10 70
doing 10 80
doing 10 90
doing 10 100
doing 10 110
doing 10 120
doing 10 130
doing 10 140
doing 10 150
doing 10 160
doing 10 170
doing 10 180
doing 10 190
doing 10 201
done 10 0.20567584037780762
doing 20 0
doing 20 10
doing 20 20
doing 20 30
doing 20 40
doing 20 50
doing 20 60
doing 20 70
doing 20 80
doing 20 90
doing 20 100
doing 20 110
doing 20 120
doing 20 130
doing 20 140
doing 20 150
doing 20 160
doing 20 170
doing 20 180
doing 20 190
doing 20 201
done 20 0.19366455078125
doing 30 0
doing 30 10
doing 30 20
doing 30 30
doing 30 40
doing 30 50
doing 30 60
doing 30 70
doing 30 80
doing 30 90
doing 30 100
doing 30 110
doing 30 1

In [54]:
xx=z1[:,:,0]
# xx.transpose(1,0,2).shape

In [55]:
xx.shape

(201, 201)

In [53]:
np.all(xx==xx.T)

True

In [ ]:
crash

In [ ]:
m1=-2
m2=2
m3=0

arrs=[da.hstack([da.vstack([da.from_delayed(delayed(wig3j_recur_1d)(l[i],l[k],m1,m2,m3,
                                                               wlmax),
                    shape=(1,1,wlmax),dtype='float32') 
                    for i in np.arange(lmax+1)]) 
                     for k in np.arange(lmax+1)])
                      ]
arrs2=da.concatenate(arrs,axis=-1)
# arrs2=arrs2.rechunk(chunks=(int(lmax/1),int(lmax/1),int(wlmax)))

print(fname.format(i=str(2)+'j3m',lmax=lmax,wlmax=wlmax))
%time arrs2.to_zarr(fname.format(i=str(2)+'j3m',lmax=lmax,wlmax=wlmax),overwrite=True)

In [ ]:
arrs2.shape

In [ ]:
crash

In [ ]:
l

In [ ]:
arrs2.shape

In [ ]:
m1=-2
m2=2
m3=0

arrs=[da.hstack([da.vstack([da.from_delayed(delayed(wig3j_map)(m1,m2,m3,l[lb[i]:lb[i+1]],l[lb[k]:lb[k+1]],
                                                               np.atleast_1d(w_l[j3_b[j3]:j3_b[j3+1] ]),asym_fact),
                    shape=(lb[i+1]-lb[i],lb[k+1]-lb[k],j3_b[j3+1]-j3_b[j3]),
                                            dtype='float32') 
                    for i in np.arange(len(lb)-1)]) 
                     for k in np.arange(len(lb)-1)])
                      for j3 in np.arange(len(j3_b)-1)]
arrs2=da.concatenate(arrs,axis=-1)
arrs2=arrs2.rechunk(chunks=(lmax/10,lmax/10,wlmax/10))
# print(fname.format(i=str(2)+'j3m',lmax=lmax,wlmax=wlmax))
# %time arrs2.to_zarr(fname.format(i=str(2)+'j3m',lmax=lmax,wlmax=wlmax),overwrite=True)

In [ ]:
arrs2.shape

In [ ]:
client.restart()

In [ ]:
m1=-2
m2=2
m3=0

arrs=[da.hstack([da.vstack([da.from_delayed(delayed(wig3j_map)(m1,m2,m3,l[lb[i]:lb[i+1]],l[lb[k]:lb[k+1]],
                                                               np.atleast_1d(j3),asym_fact),
                    shape=(lb[i+1]-lb[i],lb[k+1]-lb[k],1),dtype='float32') 
                    for i in np.arange(len(lb)-1)]) 
                     for k in np.arange(len(lb)-1)])
                      for j3 in w_l]
# arrs2=da.stack(arrs)
arrs2=da.concatenate(arrs,axis=-1)
arrs2=arrs2.rechunk(chunks=(lmax/10,lmax/10,wlmax/10))
%time arrs2.to_zarr(fname.format(i=2,lmax=lmax,wlmax=wlmax),overwrite=True)

In [ ]:
crash

In [ ]:
arrs2=zarr.open(fname.format(i=2,lmax=lmax,wlmax=wlmax))

In [ ]:
arrs2.oindex[0,:,:].max()

In [ ]:
z1=zarr.open('./temp/dask_wig3j_l100_w30_0_asym50.zarr/')
z2=np.array(zarr.open('./temp/dask_wig3j_l100_w30_0.zarr/'))

In [ ]:
z1.reshape(1,3000,100)

In [ ]:
crash

In [ ]:
zaa_in=zarr.open('temp/dask_test0.zarr')

In [ ]:
np.einsum('i,ijk->jk',w_l,zaa_in)

In [ ]:
zaa_in.oindex[np.int32(w_l[:5]), [1, 3],[1,5]]

In [ ]:
zaa_in

In [ ]:
aa2 = da.from_array(zaa_in, chunks=zaa_in.chunks)

In [ ]:
aa2

In [ ]:
m_1=0
m_2=0
m_3=0
fname='temp/wigner_test_big.h5'

with h5py.File(fname,'w') as f:
    dst = f.create_dataset("0", shape=(lmax,lmax,wlmax),
                           dtype=np.float32)
    lm=0
    while lm<lmax:
        l_t=np.arange(lm,lm+l_step)
        lm2=0
        while lm2<lmax:
            if lm2>lm+l_step+wlmax or lm>lm2+l_step+wlmax:
                print(lm2,lm)
                lm2+=l_step
                continue
            l_t2=np.arange(lm2,lm2+l_step)
            dst[lm:lm+l_step,lm2:lm2+l_step,:]=Wigner3j_parallel( m_1, m_2, m_3,l_t, l_t2 , w_l,ncpu=ncpu)
            
            lm2+=l_step
            
        lm+=l_step

In [ ]:
f.close()

In [ ]:
m_1=2
m_2=-2
m_3=0

with h5py.File(fname,'a') as f:
    dst = f.create_dataset("2", shape=(lmax,lmax,wlmax),
                           dtype=np.float32)
    lm=0
    while lm<lmax:
        l_t=np.arange(lm,lm+l_step)
        lm2=0
        while lm2<lmax:
            if lm2>lm+l_step+wlmax or lm>lm2+l_step+wlmax:
                print(lm2,lm)
                lm2+=l_step
                continue
            l_t2=np.arange(lm2,lm2+l_step)
            dst[lm:lm+l_step,lm2:lm2+l_step,:]=Wigner3j_parallel( m_1, m_2, m_3,l_t, l_t2 , w_l,ncpu=ncpu)
            lm2+=l_step
        lm+=l_step

In [ ]:
f.close()

In [ ]:
fname='temp/wigner_test.h5'

In [ ]:
f = h5py.File(fname, 'r')


In [ ]:
d=f["0"]

In [ ]:
d.shape

In [ ]:
w=np.ones_like(w_l)

In [ ]:
M=np.zeros((500,500))
M2=np.zeros((500,500))
M[:,:]=np.dot(d,w)

In [ ]:
x=d[w_l[3:5],:,:]#

In [ ]:
x[:,np.int32(w_l[:5]),:]

In [ ]:
lm=0
while lm<lmax:
    M2[lm:lm+l_step,:]=np.dot(d[lm:lm+l_step,:,:],w)
    lm+=l_step

In [ ]:
np.all(M2==M)

In [ ]:
f.close()

In [ ]:
coo_matrix((100,100))